In [19]:
from pathlib import Path
import argparse
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from __future__ import annotations
from pathlib import Path
import pandas as pd
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
)
from sklearn.metrics import (
    precision_recall_fscore_support,
    accuracy_score,
)

In [20]:
def load_labels_csv(csv_path: Path) -> pd.DataFrame:
    """GT: frame 列＋行動列 (one-hot) の DataFrame[frame]"""
    df = pd.read_csv(csv_path)
    df = df.loc[:, ~df.columns.str.contains(r"^Unnamed")]
    if "frame" not in df.columns:
        df.insert(0, "frame", df.index)
    return df.set_index("frame")

In [21]:
def load_pred_csv(csv_path: Path, behaviors: list[str]) -> pd.DataFrame:
    """予測 CSV → one-hot DataFrame[frame] (behaviors 列のみ)"""
    df = pd.read_csv(csv_path)

    # one-hot 形式ならそのまま
    if set(behaviors).issubset(df.columns):
        if "frame" not in df.columns:
            df.insert(0, "frame", df.index)
        return df.set_index("frame")[behaviors]

    # frame + class_name 形式を pivot
    if {"frame", "class_name"}.issubset(df.columns):
        pivot = (
            df.assign(flag=1)
            .pivot_table(index="frame", columns="class_name",
                         values="flag", fill_value=0)
            .reindex(columns=behaviors, fill_value=0)
        )
        return pivot

    raise ValueError(f"{csv_path} は既知の形式ではありません。")

In [22]:
# ────────────────────────────────────────────────────────────────
# ❷  メトリクス計算
# ────────────────────────────────────────────────────────────────
def calc_metrics(gt: pd.DataFrame, pr: pd.DataFrame,
                 behaviors: list[str]) -> pd.DataFrame:
    """各行動列の accuracy / precision / recall / f1"""
    rows = []
    for beh in behaviors:
        y_true, y_pred = gt[beh], pr[beh]
        prec, rec, f1, _ = precision_recall_fscore_support(
            y_true, y_pred, average="binary", zero_division=0
        )
        acc = accuracy_score(y_true, y_pred)
        rows.append(
            dict(behavior=beh,
                 accuracy=acc,
                 precision=prec,
                 recall=rec,
                 f1_score=f1)
        )
    return pd.DataFrame(rows)

In [23]:
# ----------------------------------------------------------------------
# ❸  エントリーポイント
# ----------------------------------------------------------------------
def main(list_csv: str = "../data/file_list_train.csv", out_csv: str = "../outputs/metrics_summary_train.csv") -> None:
    pairs = pd.read_csv(list_csv)
    all_results = []          # 各動画のメトリクス
    all_gt, all_pr = [], []   # 全動画連結用

    for _, row in pairs.iterrows():
        pred_path = Path(f"../data/YORU_data/{row[0]}")
        gt_path = Path(f"../data/BORIS_data/{row[1]}")

        if not (pred_path.exists() and gt_path.exists()):
            print(f"⚠️  Skipped: {pred_path} / {gt_path} が見つかりません。")
            continue

        gt_df = load_labels_csv(gt_path)
        behaviors = list(gt_df.columns)
        pr_df = load_pred_csv(pred_path, behaviors)

        # 共通フレームで揃える
        common = gt_df.index.intersection(pr_df.index)
        gt_df, pr_df = gt_df.loc[common], pr_df.loc[common]

        # 動画単体のメトリクス
        metrics_df = calc_metrics(gt_df, pr_df, behaviors)
        metrics_df.insert(0, "video", pred_path.stem)
        metrics_df.insert(1, "n_frames", len(common))
        all_results.append(metrics_df)

        # 全体計算用に蓄積
        all_gt.append(gt_df)
        all_pr.append(pr_df)

    # ---------- 保存 ----------
    if not all_results:
        print("❌ 計算対象がありません。")
        return

    summary = pd.concat(all_results, ignore_index=True)

    # ----------  全体 (ALL) 行を追加 ----------
    #   すべての動画を縦連結して改めて計算
    concat_gt = pd.concat(all_gt).sort_index()
    concat_pr = pd.concat(all_pr).sort_index()
    all_metrics = calc_metrics(concat_gt, concat_pr, behaviors)
    all_metrics.insert(0, "video", "ALL")
    all_metrics.insert(1, "n_frames", len(concat_gt))
    summary = pd.concat([summary, all_metrics], ignore_index=True)

    summary.to_csv(out_csv, index=False)
    print(f"✅  {out_csv} を保存しました。")


In [24]:
main(list_csv = "../data/file_list_train.csv", out_csv ="../outputs/metrics_summary_train.csv")
main(list_csv = "../data/file_list_test.csv", out_csv ="../outputs/metrics_summary_test.csv")

/var/folders/5p/x5rm7nzs41bdcd02r6p1ht540000gn/T/ipykernel_16473/1862066230.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred_path = Path(f"../data/YORU_data/{row[0]}")
/var/folders/5p/x5rm7nzs41bdcd02r6p1ht540000gn/T/ipykernel_16473/1862066230.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gt_path = Path(f"../data/BORIS_data/{row[1]}")
/var/folders/5p/x5rm7nzs41bdcd02r6p1ht540000gn/T/ipykernel_16473/1862066230.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To acce

✅  ../outputs/metrics_summary_train.csv を保存しました。


/var/folders/5p/x5rm7nzs41bdcd02r6p1ht540000gn/T/ipykernel_16473/1862066230.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred_path = Path(f"../data/YORU_data/{row[0]}")
/var/folders/5p/x5rm7nzs41bdcd02r6p1ht540000gn/T/ipykernel_16473/1862066230.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gt_path = Path(f"../data/BORIS_data/{row[1]}")
/var/folders/5p/x5rm7nzs41bdcd02r6p1ht540000gn/T/ipykernel_16473/1862066230.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To acce

✅  ../outputs/metrics_summary_test.csv を保存しました。
